# Import libraries

In [1]:
# import libraries
import random
import numpy as np
import pandas as pd
import math 
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

import scipy
from scipy.spatial.distance import pdist, squareform
from scipy.stats import chi2
import time

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager
plt.style.use(['ggplot'])

# Data preprocessing

In [2]:
# Import the dataset
# The dataset is Highway-Rail Grade Crossing Accident dataset
# Importing the dataset, Highway-Rail Grade Crossing Accident dataset
data = pd.read_csv('Highway-Rail_Grade_Crossing_Accident_Data.csv', low_memory=False)
print("Original data shape:", data.shape)

# Count accidents by unique company pairs
grouped_data = data.groupby(['Railroad Code', 'Incident Year', 'State Name', 'Highway User Position', 'Equipment Involved']).size().reset_index(name='Accident Count')
print("Grouped data shape:", grouped_data.shape)

# Merge the 'Accident Count' column back into the original data
merged_data = pd.merge(data, grouped_data, on=['Railroad Code', 'Incident Year', 'State Name', 'Highway User Position', 'Equipment Involved'], how='left')
print("Merged data shape:", merged_data.shape)

# Define features and target variable
categorical_features = ['Highway User Position', 'Equipment Involved']
numeric_features = ['Incident Year']
target = 'Accident Count'

# Drop rows with missing values in the target variable
merged_data.dropna(subset=[target], inplace=True)

# Convert 'Incident Year' to full year format
merged_data['Incident Year'] = merged_data['Incident Year'].apply(lambda year: 2000 + year if year <= 21 else 1900 + year)

# Define X and y
X = merged_data[numeric_features + categorical_features]
y = merged_data[target]
y = np.reshape(y, (len(y), 1))

# Convert categorical features to numerical data
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_features)], remainder='passthrough')
X_transformed = ct.fit_transform(X).toarray()

sample_size = int(X_transformed.shape[0] * 0.3)  

rows_id = random.sample(range(X_transformed.shape[0]), sample_size)
X_1 = X_transformed[rows_id, :]
y1 = y[rows_id]

print("The size of X_1 after sampling is {}".format(X_1.shape))
print("The size of y1 after sampling is {}".format(y1.shape))

Original data shape: (239487, 141)
Grouped data shape: (52706, 6)
Merged data shape: (239487, 142)
The size of X_1 after sampling is (71725, 19)
The size of y1 after sampling is (71725, 1)


In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1, y1, test_size = 0.20, random_state = 0)
print("The size of training data is {}".format(X_train.shape))
print("The size of test data is {}".format(X_test.shape))

The size of training data is (57380, 19)
The size of test data is (14345, 19)


In [4]:
# Normalize the data (fitting the scaler on the training data only)
scaler_X = MinMaxScaler().fit(X_train)
scaler_y = MinMaxScaler().fit(y_train)
X_train_scaled = scaler_X.transform(X_train)
y_train_scaled = scaler_y.transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

# Train model

In [5]:
train_X, train_Y = X_train_scaled.T, y_train_scaled.T
test_X, test_Y = X_test_scaled.T, y_test_scaled.T

In [6]:
k = 10  # number of worker nodes
kf = KFold(n_splits=k, shuffle=True, random_state=42)
splits = list(kf.split(X_train_scaled))

In [7]:
def rf_map(X, L, sigma):
    # L:  number of random features
    np.random.seed(1)
    d,_ = X.shape
    #generate multivariate normal distribution
    Mu = np.zeros(d)     # zero mean     
    Sigma = 1/(sigma**2) * np.eye(d)  # covariance matrix
    Omega = np.random.multivariate_normal(Mu, Sigma, L)
    b = np.random.uniform(0, 2*np.pi, (L, 1))
    Phi_X = math.sqrt(2/L) * np.cos(np.dot(Omega, X) + b)
    return Phi_X, Omega, b

In [8]:
def compute_cost(yhat, y):
    """
    yhat is the output from fully connected layer (num_classes*num_examples)
    y is labels (num_classes*num_examples, y is one-hot encoded vector)
    """
    m = y.shape[1]  #num_examples
    diff  = y - yhat
    loss = (1/2/m) * np.sum(np.square(diff))   
    return loss

In [9]:
# Define federated training function
def federated_training(X, y, Phi_train, lr, lambd, rounds, splits):
    theta_global = np.zeros((Phi_train.shape[0], 1))
    for t in range(rounds):
        gradient_accumulator = np.zeros_like(theta_global)
        for train_index, test_index in splits:
            X_local_train, y_local_train = X[:, train_index], y[:, train_index]
            X_local_test, y_local_test = X[:, test_index], y[:, test_index]

            Phi_local_train = Phi_train[:, train_index]
            Phi_local_test = Phi_train[:, test_index]

            # Local model update through gradient descent
            predictions = np.dot(theta_global.T, Phi_local_train)
            error = predictions - y_local_train
            gradient = np.dot(Phi_local_train, error.T) / len(train_index) + lambd * theta_global
            theta_global -= lr * gradient

            # Accumulate gradients
            gradient_accumulator += gradient

        # Average gradients and update global model
        theta_global -= lr * gradient_accumulator / k

    return theta_global

In [10]:
# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

# # Define the hyperparameter space
# space = {
#     'lr': hp.uniform('lr', 0.001, 0.1),
#     'lambd': hp.loguniform('lambd', np.log(1e-5), np.log(1e-3)),
#     'rounds': hp.choice('rounds', range(5, 20)),
#     'L': hp.choice('L', range(100, 400)),
#     'sigma': hp.choice('sigma', range(1, 6))
# }

# def objective(params):
#     # Extract parameters from the Hyperopt params dictionary
#     L = int(params['L'])
#     sigma = params['sigma']
#     Phi_train, Omega, b = rf_map(train_X, L, sigma)
#     Phi_test, Omega_test, b_test = rf_map(test_X, L, sigma)
    
#     lr = params['lr']
#     lambd = params['lambd']
#     rounds = int(params['rounds'])
#     k = 10  # number of splits
#     kf = KFold(n_splits=k, shuffle=True, random_state=42)
#     splits = list(kf.split(X_train_scaled))
    
#     # Perform federated training (assuming federated_training function is defined)
#     theta_global = federated_training(train_X, train_Y, Phi_train, lr, lambd, rounds, splits)
    
#     # Evaluate the model (assuming a suitable evaluation metric function is defined)
#     testY_RF = np.dot(theta_global.T, Phi_test)

#     # Calculate test MSE and RMSE
#     test_mse = compute_cost(testY_RF, test_Y)
#     test_rmse = math.sqrt(test_mse)
    
#     performance_metric = test_rmse

#     # Return the loss value and status
#     return {'loss': performance_metric, 'status': STATUS_OK}

# # Perform Bayesian optimization
# trials = Trials()
# best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

# # Print the best hyperparameters
# print("Best Hyperparameters:", best_params)

In [11]:
# RF mapping performed once
L = 275
sigma = 1
Phi_train, Omega, b = rf_map(train_X, L, sigma)
Phi_test, Omega_test, b_test = rf_map(test_X, L, sigma)

# Federated learning parameters
lr = 0.09584131766022012
lambd = 0.0003648662609891207
rounds = 14
k = 10  # number of splits
kf = KFold(n_splits=k, shuffle=True, random_state=42)
splits = list(kf.split(X_train_scaled))

# Perform federated training
start = time.perf_counter()
theta_global = federated_training(train_X, train_Y, Phi_train, lr, lambd, rounds, splits)
end = time.perf_counter()
trf = end - start
# print("The process takes %.4f seconds " %(trf))
# print("Train cost is " + str(train_costs_RF[-1]))
# print("Test MSE is " + str(test_MSE_RF[-1]))
# print("Test RMSE is " + str(test_rmse))

In [12]:
# Evaluate the global model using test set
testY_RF = np.dot(theta_global.T, Phi_test)

# Calculate test MSE and RMSE
test_mse = compute_cost(testY_RF, test_Y)
test_rmse = math.sqrt(test_mse)

In [13]:
# Output the results
print("The process takes %.4f seconds " %(trf))
print(f"Test MSE is {test_mse}")
print(f"Test RMSE is {test_rmse}")

The process takes 6.8368 seconds 
Test MSE is 0.007540100465111884
Test RMSE is 0.08683375187743464


In [14]:
# Paper main idea

# instead of transforming the data into a higher dimensional space, we use random features 
# that capture the important aspects of the data and transform the data into a smaller 
# dimensional space to reduce computational complexity.